# Welcome to your first assignment!

Instructions are below. Please give this a try, and look in the solutions folder if you get stuck (or feel free to ask me!)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">Just before we get to the assignment --</h2>
            <span style="color:#f71;">I thought I'd take a second to point you at this page of useful resources for the course. This includes links to all the slides.<br/>
            <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">https://edwarddonner.com/2024/11/13/llm-engineering-resources/</a><br/>
            Please keep this bookmarked, and I'll continue to add more useful links there over time.
            </span>
        </td>
    </tr>
</table>

# HOMEWORK EXERCISE ASSIGNMENT

Upgrade the day 1 project to summarize a webpage to use an Open Source model running locally via Ollama rather than OpenAI

You'll be able to use this technique for all subsequent projects if you'd prefer not to use paid APIs.

**Benefits:**
1. No API charges - open-source
2. Data doesn't leave your box

**Disadvantages:**
1. Significantly less power than Frontier Model

## Recap on installation of Ollama

Simply visit [ollama.com](https://ollama.com) and install!

Once complete, the ollama server should already be running locally.  
If you visit:  
[http://localhost:11434/](http://localhost:11434/)

You should see the message `Ollama is running`.  

If not, bring up a new Terminal (Mac) or Powershell (Windows) and enter `ollama serve`  
And in another Terminal (Mac) or Powershell (Windows), enter `ollama pull llama3.2`  
Then try [http://localhost:11434/](http://localhost:11434/) again.

If Ollama is slow on your machine, try using `llama3.2:1b` as an alternative. Run `ollama pull llama3.2:1b` from a Terminal or Powershell, and change the code below from `MODEL = "llama3.2"` to `MODEL = "llama3.2:1b"`

In [ ]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display

In [ ]:
# Constants

OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
MODEL = "llama3.2"

In [ ]:
# Create a messages list using the same format that we used for OpenAI

messages = [
    {"role": "user", "content": "Describe some of the business applications of Generative AI"}
]

In [ ]:
payload = {
        "model": MODEL,
        "messages": messages,
        "stream": False
    }

In [ ]:
# Let's just make sure the model is loaded

!ollama pull llama3.2

In [ ]:
# If this doesn't work for any reason, try the 2 versions in the following cells
# And double check the instructions in the 'Recap on installation of Ollama' at the top of this lab
# And if none of that works - contact me!

response = requests.post(OLLAMA_API, json=payload, headers=HEADERS)
print(response.json()['message']['content'])

# Introducing the ollama package

And now we'll do the same thing, but using the elegant ollama python package instead of a direct HTTP call.

Under the hood, it's making the same call as above to the ollama server running at localhost:11434

In [ ]:
import ollama

response = ollama.chat(model=MODEL, messages=messages)
print(response['message']['content'])

## Alternative approach - using OpenAI python library to connect to Ollama

In [ ]:
# There's actually an alternative approach that some people might prefer
# You can use the OpenAI client python library to call Ollama:

from openai import OpenAI
ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

response = ollama_via_openai.chat.completions.create(
    model=MODEL,
    messages=messages
)

print(response.choices[0].message.content)

## Are you confused about why that works?

It seems strange, right? We just used OpenAI code to call Ollama?? What's going on?!

Here's the scoop:

The python class `OpenAI` is simply code written by OpenAI engineers that makes calls over the internet to an endpoint.  

When you call `openai.chat.completions.create()`, this python code just makes a web request to the following url: "https://api.openai.com/v1/chat/completions"

Code like this is known as a "client library" - it's just wrapper code that runs on your machine to make web requests. The actual power of GPT is running on OpenAI's cloud behind this API, not on your computer!

OpenAI was so popular, that lots of other AI providers provided identical web endpoints, so you could use the same approach.

So Ollama has an endpoint running on your local box at http://localhost:11434/v1/chat/completions  
And in week 2 we'll discover that lots of other providers do this too, including Gemini and DeepSeek.

And then the team at OpenAI had a great idea: they can extend their client library so you can specify a different 'base url', and use their library to call any compatible API.

That's it!

So when you say: `ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')`  
Then this will make the same endpoint calls, but to Ollama instead of OpenAI.

## Also trying the amazing reasoning model DeepSeek

Here we use the version of DeepSeek-reasoner that's been distilled to 1.5B.  
This is actually a 1.5B variant of Qwen that has been fine-tuned using synethic data generated by Deepseek R1.

Other sizes of DeepSeek are [here](https://ollama.com/library/deepseek-r1) all the way up to the full 671B parameter version, which would use up 404GB of your drive and is far too large for most!

In [ ]:
!ollama pull deepseek-r1:1.5b

In [ ]:
# This may take a few minutes to run! You should then see a fascinating "thinking" trace inside <think> tags, followed by some decent definitions

response = ollama_via_openai.chat.completions.create(
    model="deepseek-r1:1.5b",
    messages=[{"role": "user", "content": "Please give definitions of some core concepts behind LLMs: a neural network, attention and the transformer"}]
)

print(response.choices[0].message.content)

# NOW the exercise for you

Take the code from day1 and incorporate it here, to build a website summarizer that uses Llama 3.2 running locally instead of OpenAI; use either of the above approaches.

In [ ]:
# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"

In [ ]:
from ollama import pull


!ollama pull llama3.2:latest

In [ ]:
def process_contact_info(scrape_data):
    """
    Extracts and summarizes contact details from the website scrape.

    Args:
        scrape_data (dict): Website scrape JSON data.

    Returns:
        dict: Structured contact information summary.
    """
    contact_info = scrape_data.get('contact_info', {})

    # Case: no contact info or no emails
    if not contact_info or (
        not contact_info.get('emails')
        and not contact_info.get('phones')
        and not contact_info.get('addresses_guess')
    ):
        return {
            'contact_page_exists': False,
            'contact_page_url': scrape_data.get('contact_link'),
            'emails': [],
            'phones': [],
            'addresses_guess': [],
            'notes': "Only contact form or no visible contact info"
        }

    # Prepare structured output
    return {
        'contact_page_exists': True,
        'contact_page_url': scrape_data.get('contact_link'),
        'emails': contact_info.get('emails', []),
        'phones': contact_info.get('phones', []),
        'addresses_guess': contact_info.get('addresses_guess', []),
        'notes': None if contact_info.get('emails') else "Only contact form"
    }

In [ ]:
# JUPYTER-READY SELENIUM SCRAPER (robust headless + fallback)
# pip install selenium webdriver-manager  # uncomment this line to install the dependencies

import re, time, json
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import WebDriverException, TimeoutException

EMAIL_RE = re.compile(r'[A-Z0-9._%+-]+@[A-Z0-9.-]+\.[A-Z]{2,}', re.I)
PHONE_RE = re.compile(r'\+?\d[\d\-\s().]{7,}\d')
ADDRESS_HINTS = re.compile(r'\b(street|st\.|ave|avenue|road|rd\.|suite|ste\.|blvd|way|lane|ln\.|zip|postal|city|state|country)\b', re.I)
CONTACT_HINTS = [r'\bcontact\b', r'\bcontact-us\b', r'\bcontactus\b', r'\bsupport\b', r'\bget-in-touch\b', r'\bhelp\b']

def _make_driver(headless=True, page_load_strategy="eager"):
    opts = webdriver.ChromeOptions()
    if headless:
        # More stable headless in Jupyter
        opts.add_argument("--headless=new")
        opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--window-size=1400,900")
    opts.add_argument("--enable-features=NetworkService,NetworkServiceInProcess")
    opts.add_argument("--disable-features=IsolateOrigins,site-per-process")
    # Desktop UA helps with some bot/CDN setups
    opts.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/125.0.0.0 Safari/537.36")
    opts.page_load_strategy = page_load_strategy  # "eager" is less crashy than "normal"

    svc = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=svc, options=opts)
    driver.set_page_load_timeout(25)
    return driver

def _safe_wait(driver, timeout=15):
    WebDriverWait(driver, timeout).until(
        EC.any_of(
            EC.presence_of_all_elements_located((By.TAG_NAME, "a")),
            EC.presence_of_all_elements_located((By.TAG_NAME, "nav")),
            EC.presence_of_all_elements_located((By.TAG_NAME, "footer")),
            EC.presence_of_all_elements_located((By.TAG_NAME, "p")),
        )
    )

def _click_cookie_banner_if_present(driver):
    try:
        buttons = driver.find_elements(By.TAG_NAME, "button")
        texts = ("accept", "agree", "ok", "got it", "continue")
        for b in buttons:
            lbl = " ".join([(b.text or ""), (b.get_attribute("aria-label") or ""), (b.get_attribute("data-testid") or "")]).lower()
            if any(t in lbl for t in texts):
                try:
                    b.click()
                    time.sleep(0.3)
                    break
                except Exception:
                    pass
    except Exception:
        pass

def _navigate(driver, url, retries=2):
    last_err = None
    for attempt in range(retries + 1):
        try:
            driver.get(url)
            _safe_wait(driver, timeout=20)
            return True
        except (TimeoutException, WebDriverException) as e:
            last_err = e
            time.sleep(0.8)
    raise last_err

def _find_contact_link(driver, base_url):
    WebDriverWait(driver, 12).until(EC.presence_of_all_elements_located((By.TAG_NAME, "a")))
    anchors = driver.find_elements(By.TAG_NAME, "a")
    host = urlparse(base_url).netloc
    scored = []
    for a in anchors:
        try:
            href = (a.get_attribute("href") or "").strip()
            if not href:
                continue
            txt = (a.text or "")
            aria = (a.get_attribute("aria-label") or "")
            cls  = (a.get_attribute("class") or "")
            blob = " ".join([txt, href, aria, cls]).lower()
            s = 0
            for pat in CONTACT_HINTS:
                if re.search(pat, blob):
                    s += 2
            # internal link bonus
            if urlparse(href).netloc in ("", host):
                s += 1
            # footer-ish bonus
            try:
                if a.location and a.location.get("y", 0) > 600:
                    s += 1
            except Exception:
                pass
            if s > 0:
                scored.append((s, href))
        except Exception:
            pass
    if not scored:
        for guess in ("/contact", "/contact-us", "/support", "/about#contact"):
            scored.append((1, urljoin(base_url, guess)))
    scored.sort(key=lambda x: x[0], reverse=True)
    return scored[0][1] if scored else None

def _safe_page_text(html, max_chars=800_000, max_lines=50_000):
    html = html[:max_chars]
    soup = BeautifulSoup(html, "html.parser")
    parts = []
    for i, s in enumerate(soup.stripped_strings):
        if i >= max_lines:
            break
        parts.append(s)
    return "\n".join(parts)

def _extract_contact_info(html_text):
    text = _safe_page_text(html_text)[:500_000]
    emails = sorted(set(EMAIL_RE.findall(text)))
    phones = sorted(set(p.strip() for p in PHONE_RE.findall(text)))
    lines = text.splitlines()
    addresses = []
    for i, line in enumerate(lines[:5000]):
        if ADDRESS_HINTS.search(line):
            block = "\n".join(lines[max(0, i-2):min(len(lines), i+3)])
            if block not in addresses:
                addresses.append(block)
        if len(addresses) >= 3:
            break
    return {"emails": emails, "phones": phones, "addresses_guess": addresses}

def get_contact_details(url, prefer_headless=True):
    """
    Robust entrypoint for Jupyter:
    - Try headless first (fast), retry navigate
    - If it crashes, auto fallback to non-headless
    """
    # 1) Try headless
    try:
        driver = _make_driver(headless=prefer_headless, page_load_strategy="eager")
        try:
            _navigate(driver, url, retries=2)
            _click_cookie_banner_if_present(driver)
            contact_href = _find_contact_link(driver, url)
            result = {"start_url": url, "contact_link": contact_href, "headless": True}
            if not contact_href:
                result["error"] = "No contact link found"
                return result
            if contact_href.startswith("mailto:"):
                result["contact_info"] = {"emails": [contact_href.replace("mailto:", "")], "phones": [], "addresses_guess": []}
                return result
            _navigate(driver, contact_href, retries=2)
            _click_cookie_banner_if_present(driver)
            html = driver.page_source
            result["contact_info"] = _extract_contact_info(html)
            return result
        finally:
            driver.quit()
    except Exception as headless_err:
        # 2) Fallback to non-headless (interactive) if headless crashes
        try:
            driver = _make_driver(headless=False, page_load_strategy="eager")
            try:
                _navigate(driver, url, retries=2)
                _click_cookie_banner_if_present(driver)
                contact_href = _find_contact_link(driver, url)
                result = {"start_url": url, "contact_link": contact_href, "headless": False, "headless_error": str(headless_err)}
                if not contact_href:
                    result["error"] = "No contact link found"
                    return result
                if contact_href.startswith("mailto:"):
                    result["contact_info"] = {"emails": [contact_href.replace("mailto:", "")], "phones": [], "addresses_guess": []}
                    return result
                _navigate(driver, contact_href, retries=2)
                _click_cookie_banner_if_present(driver)
                html = driver.page_source
                result["contact_info"] = _extract_contact_info(html)
                return result
            finally:
                driver.quit()
        except Exception as non_headless_err:
            return {
                "start_url": url,
                "error": "Navigation failed in both headless and non-headless modes",
                "headless_error": str(headless_err),
                "non_headless_error": str(non_headless_err),
            }
# --------- Optional: LLM summary wrapper ----------
def summarize_contact_with_gpt(url, prefer_headless=True, model="gpt-5-mini"):
    data = get_contact_details(url, prefer_headless=prefer_headless)
    structured_info = process_contact_info(data)
    print(structured_info)

    # If scraping failed, return a concise message (or raise)
    if isinstance(structured_info, dict) and structured_info.get("error"):
        return f"Scrape failed for {url}:\n{structured_info}"

    # # If OpenAI isn't available, just return a plain summary string
    # if not _HAS_OPENAI:
    #     return f"(No OpenAI client) Contact summary for {url}:\n{data}"

    # client = OpenAI()
    system_prompt = (
        "You extract and summarize contact details (emails, phones, addresses) "
        "from a website scrape. Be concise and structured."
    )
    user_prompt = f"""Website: {url}

Raw scrape JSON:
{structured_info}

Return:
- Whether contact page exists and its URL
- Emails (list)
- Phones (list)
- Address guess (1-2 lines if present)
- Any notes (e.g., only contact form, no email)
"""
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

    resp = ollama_via_openai.chat.completions.create(
    model="llama3.2:latest",
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
)


    return resp.choices[0].message.content

In [ ]:
summary = summarize_contact_with_gpt("https://www.denvermarketinggroup.com", prefer_headless=True)
display(Markdown(summary))